In [3]:

import numpy as np
import pandas as pd
import re
import csv
import os
import pickle

from _datetime import datetime
from sklearn.preprocessing import OneHotEncoder

In [4]:

product_headers_to_encode = ['PRODUCTTYPENAME', 'ROUTENAME', 'DOSAGEFORMNAME', 'MARKETINGCATEGORYNAME', 'ACTIVE_NUMERATOR_STRENGTH', 'ACTIVE_INGRED_UNIT']
package_headers_to_encode = ['PACKAGEUNIT', 'PACKAGETYPE']

target_encoding = 'utf-8'
separ = '\t'
custom_sep = ' ?[|,;:<>] ?|^ | $'

product_file = 'Product2.csv'
package_file = 'Package2.csv'

encoder_dir = 'encoders/'
encoding_dir = 'enconding_dic/'

encoded_product_file = 'transformed_product_data.csv'
encoded_package_file = 'transformed_package_data.csv'

product_encode_file_exist = False
package_encode_file_exist = False


# TODO incohernce entre dates
# TODO incohernce entre routname / forme
# TODO incohernce entre valeurs numeric abberantes (ordre de grandeur)
# TODO incohernce entre valeurs phase et l'emballage
# TODO tester imputatin itérative

def assert_table_completeness(table):
    empty_cells = table.shape[0] - table.count(axis=0)
    unique_values = table.nunique(axis=0)

    print('Empty cells:\n{}\n'.format(empty_cells))
    print('Unique values:\n{}\n'.format(unique_values))


def assert_product_id_completeness(table, header):
    empty_cells = table.shape[0] - table.count(axis=0)
    unique_values = table.nunique(axis=0)

    if empty_cells[header] == 0:
        print('No empty values in the {} column'.format(header))
    else:
        print('There are {} empty values in the {} column'.format(empty_cells[header], header))

    if unique_values[header] == table.shape[0]:
        print('No duplicat values in the {} column'.format(header))
    else:
        print('There are {} duplicat values in the {} column\n\n'.format(table.shape[0] - unique_values[header], header))


def get_unique_values(table, headers=''):
    uniques = {}
    if headers == '':
        cols = table.columns.values
        for n, c in enumerate(cols):
            uniques[c] = pd.unique(table[c])
    elif type(headers) is list:
        for header in headers:
            uniques[header] = pd.unique(table[header])
    elif type(headers) is str:
        uniques[headers] = pd.unique(table[headers])
    return uniques


def df_to_lower(table, columns='all'):
    cols = table.columns.values if columns == 'all' else columns
    for c in cols:
        try:
            table[c] = table[c].str.lower()
        except:
            pass


def get_decomposed_uniques(table, header):
    decomposed_uniques = {}
    if type(header) is str:
        for unique_header, uniques in get_unique_values(table, header).items():
            tmp_lst = []
            for val in uniques:
                if type(val) is str:
                    for decomposed in re.split(custom_sep, val):
                        if decomposed != '' and not decomposed in tmp_lst:
                            tmp_lst.append(decomposed)

            tmp_lst.sort()
            decomposed_uniques[unique_header] = tmp_lst
    else:
        raise TypeError('header should be a string representing a column header')

    return pd.DataFrame.from_dict(decomposed_uniques)


def get_onehot_encoders(table, cols):
    encoder_dict = {}
    for col in cols:
        uniques_vals = get_decomposed_uniques(table, header=col)
        enc = OneHotEncoder(handle_unknown='ignore')
        enc.fit_transform(uniques_vals)
        encoder_dict[col] = enc
    return encoder_dict


def onehot_encode(table, header):
    # Create onehot codes for the specidfied column
    lst = []
    encoder_dict = get_onehot_encoders(table, [header])

    count = 0
    for index in range(table.shape[0]):
        _tmp = np.zeros([1, len(encoder_dict[header].categories_[0])], dtype=int)
        if type(table.loc[index, header]) is str:
            for decomposed in re.split(custom_sep, table.loc[index, header]):
                _tmp |= np.int_(encoder_dict[header].transform([[decomposed]]).toarray())
            lst.append(_tmp)

        # Update loading bar
        if count == 1000:
            progress(index, table.shape[0])
            count = 0
        count += 1

    print(" -> Done", flush=True)

    # Replace dataframe column by encoded values
    table.loc[:, header] = pd.Series(lst)

    # return the encoder associated to that particular header
    return encoder_dict[header]


def time_methode(methode, status='', **kwargs):
    print('Timing {}'.format(methode.__name__))
    if status != '':
        print(status)
    start_time = datetime.now()
    print('Start time: {}'.format(start_time))
    ret = methode(**kwargs)
    end_time = datetime.now()
    print('End time: {}'.format(end_time))
    print('{} took: {}'.format(methode.__name__, (end_time - start_time)))
    if ret != '':
        return ret
    else:
        ret = 0
    return ret


def progress(count, total, status=''):
    bar_len = 50
    filled_len = int(round(bar_len * count / float(total)))
    _str = ''
    percents = np.ceil(100.0 * count / float(total))
    bar = '=' * filled_len + ':' * (bar_len - filled_len)

    if status == '':
        _str = '|{}| {}%'.format(bar, percents)
    else:
        _str = '|{}| {}% - {}'.format(bar, percents, status)

    print('\r', end='', flush=True)
    print(_str, end='', flush=True)


"""
Load data:
    Will look for existing files to deserialize prior encoding data. If the files are not found
    it will proceed with the original data through encoding.
"""
product_encode_file_exist = os.path.isfile(encoded_product_file)
package_encode_file_exist = os.path.isfile(encoded_package_file)

enc_dic = {}

original_product_data = pd.read_csv(product_file, sep=';', encoding='latin1')
original_package_data = pd.read_csv(package_file, sep=';', encoding='latin1')

if product_encode_file_exist:
    print('Loading encoded product data from existing file...')
    product_data = pd.read_csv(encoded_product_file, sep=separ, encoding=target_encoding)

    # Populate onehot encoders dictionnary
    for header in product_headers_to_encode:
        enc_dic[header] = pickle.load(open(encoder_dir + '{}_data_encoder.pkl'.format(header), 'rb'))
else:
    product_data = original_product_data

if package_encode_file_exist:
    print('Loading encoded package data from existing file...')
    package_data = pd.read_csv(encoded_package_file, sep=separ, encoding=target_encoding)

    # Populate onehot encoders dictionnary
    for header in package_headers_to_encode:
        enc_dic[header] = pickle.load(open(encoder_dir + '{}_data_encoder.pkl'.format(header), 'rb'))
else:
    package_data = original_package_data

# Make everything lower characters in both tables
df_to_lower(product_data)
df_to_lower(package_data)

if product_encode_file_exist:
    print('Get unique values for ROUTENAME column of PRODUCT table')
    product_unique_values = get_decomposed_uniques(original_product_data, 'ROUTENAME')
    print(product_unique_values)
    print(enc_dic['ROUTENAME'].categories_[0])

# 1. Auscultation
## Etude des données du fichier 'package'

In [5]:
package_data.head()

,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
0,0002-0800_94c48759-29bb-402d-afff-9a713be11f0e,0002-0800,0002-0800-01,1 vial in 1 carton (0002-0800-01) > 10 ml in ...,19870710,NaN,n,n
1,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-30,"1 vial, multi-dose in 1 can (0002-1200-30) > ...",20120601,NaN,n,n
2,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-50,"1 vial, multi-dose in 1 can (0002-1200-50) > ...",20120601,NaN,n,n
3,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-61,2 syringe in 1 carton (0002-1433-61) > .5 ml ...,20141107,NaN,n,y
4,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-80,4 syringe in 1 carton (0002-1433-80) > .5 ml ...,20141107,NaN,n,n


In [6]:
print('Assessing completnes of packaging data table')
assert_table_completeness(package_data)

Assessing completnes of packaging data table
Empty cells:
PRODUCTID                  0
PRODUCTNDC              1500
NDCPACKAGECODE          2346
PACKAGEDESCRIPTION         0
STARTMARKETINGDATE         0
ENDMARKETINGDATE      167431
NDC_EXCLUDE_FLAG           0
SAMPLE_PACKAGE             0
dtype: int64

Unique values:
PRODUCTID              93084
PRODUCTNDC             91080
NDCPACKAGECODE        171447
PACKAGEDESCRIPTION    173885
STARTMARKETINGDATE      7401
ENDMARKETINGDATE         767
NDC_EXCLUDE_FLAG           1
SAMPLE_PACKAGE             2
dtype: int64



### Colonne PACKAGEDESCRIPTION
Cette colonne est présentée sous forme de phrase et contient de multiples informations. Le volume, son unité, le nombre 
de contenant et son type. S'il existe plusieurs contenants pour un objet, ils sont concaténés par un séparateur '>' de 
manière hiérarchique.

### Colonne STARTMARKETINGDATE
Les valeurs de cette colonne sont de type date. Elles ont l'air séquentielles. 

### Colonne ENDMARKETINGDATE
Les valeurs de cette colonne sont de type date. Il existe beaucoup de valeurs manquantes.

### Colonne NDC_EXCLUDE_FLAG

In [7]:
package_data['NDC_EXCLUDE_FLAG'].unique()

array(['n'], dtype=object)

Tous les objets possèdent la valeur N et ne présente aucune valeur manquante. 

### Colonne SAMPLE_PACKAGE

In [8]:
package_data['SAMPLE_PACKAGE'].value_counts()

n    173223
y       664
Name: SAMPLE_PACKAGE, dtype: int64

Les valeurs possibles sont 'Y' ou 'N'. Il y a une majorité de 'N' et aucune valeur manquante.

## Etude des données du fichier 'product'

In [9]:
product_data.head()

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,NaN,0002-0800,human otc drug,sterile diluent,NaN,diluent,"injection, solution",subcutaneous,19870710,NaN,nda,nda018781,10,water,1,ml/ml,NaN,NaN,n,20201231.0
1,NaN,0002-1200,human prescription drug,amyvid,NaN,florbetapir f 18,"injection, solution",intravenous,20120601,NaN,nda,nda202008,10,florbetapir f-18,51,mci/ml,"radioactive diagnostic agent [epc],positron em...",NaN,n,20211231.0
2,NaN,0002-1433,human prescription drug,trulicity,NaN,dulaglutide,"injection, solution",subcutaneous,20140918,NaN,bla,bla125469,10,dulaglutide,.75,mg/.5ml,"glp-1 receptor agonist [epc],glucagon-like pep...",NaN,n,20201231.0
3,NaN,0002-1434,human prescription drug,trulicity,NaN,dulaglutide,"injection, solution",subcutaneous,20140918,NaN,bla,bla125469,10,dulaglutide,1.5,mg/.5ml,"glp-1 receptor agonist [epc],glucagon-like pep...",NaN,n,20201231.0
4,NaN,0002-1436,human prescription drug,emgality,NaN,galcanezumab,"injection, solution",subcutaneous,20180927,NaN,bla,bla761063,10,galcanezumab,120,mg/ml,NaN,NaN,n,20201231.0


In [10]:
print('Assessing completnes product data table')
assert_table_completeness(product_data)

Assessing completnes product data table
Empty cells:
PRODUCTID                            1560
PRODUCTNDC                              0
PRODUCTTYPENAME                         0
PROPRIETARYNAME                         6
PROPRIETARYNAMESUFFIX               83075
NONPROPRIETARYNAME                      4
DOSAGEFORMNAME                          0
ROUTENAME                            1932
STARTMARKETINGDATE                      0
ENDMARKETINGDATE                    88915
MARKETINGCATEGORYNAME                   0
APPLICATIONNUMBER                   13097
LABELERNAME                             0
SUBSTANCENAME                        2309
ACTIVE_NUMERATOR_STRENGTH            2309
ACTIVE_INGRED_UNIT                   2309
PHARM_CLASSES                       50984
DEASCHEDULE                         88815
NDC_EXCLUDE_FLAG                        0
LISTING_RECORD_CERTIFIED_THROUGH     4325
dtype: int64

Unique values:
PRODUCTID                           91678
PRODUCTNDC                          

### Colonne PRODUCTTYPENAME

In [11]:
product_data['PRODUCTTYPENAME'].value_counts()

human otc drug                 46172
human prescription drug        44526
non-standardized allergenic     2008
plasma derivative                294
standardized allergenic          124
vaccine                          108
cellular therapy                   6
Name: PRODUCTTYPENAME, dtype: int64

Il y a 7 valeurs possibles textuelles catégorielles dans cette colonne.

### Colonne PROPRIETARYNAME

In [12]:
product_data['PROPRIETARYNAME'].nunique()

29775

In [13]:
# product_data['PROPRIETARYNAME'][393:401]

Dans cette colonne, il existe un grand nombre de valeurs différentes. Les mêmes valeurs peuvent être présentes sous 
différentes formes notamment en minuscules ou majuscules, il existe donc une inconsistance entre les valeurs.

### Colonne PROPRIETARYNAMESUFFIX

In [14]:
product_data['PROPRIETARYNAMESUFFIX'].nunique()

3726

Dans cette colonne, il y a un nombre important de valeurs manquantes. Ces valeurs sont textuelles et il existe un nombre 
élevé de valeurs différentes.

### Colonne NONPROPRIETARYNAME

In [15]:
product_data['NONPROPRIETARYNAME'].nunique()

13104

In [16]:
product_data['NONPROPRIETARYNAME'][2:6]

2     dulaglutide
3     dulaglutide
4    galcanezumab
5      ixekizumab
Name: NONPROPRIETARYNAME, dtype: object

Cette colonne présente seulement 4 valeurs manquantes. Ceux sont des données textuelles inconsistantes, par exemple 
pouvant représenter la même valeur en caratères minuscules ou majuscules. Il y a un nombre très important de valeurs 
différentes.

### Colonne DOSAGEFORMNAME

In [17]:
product_data['DOSAGEFORMNAME'].value_counts()

tablet                              15442
tablet, film coated                  9387
liquid                               8764
cream                                5104
capsule                              4824
                                    ...  
solution, gel forming / drops           1
for suspension, extended release        1
chewable gel                            1
crystal                                 1
injection, lipid complex                1
Name: DOSAGEFORMNAME, Length: 134, dtype: int64

Cette colonne contient 134 différentes valeurs textuelles. Comme on peut le voir, différentes catégories peuvent être 
affectées au même objet. La colonne ne présente aucune valeur manquante.

### Colonne ROUTENAME

In [18]:
product_data['ROUTENAME'].value_counts()

oral                                                                                54704
topical                                                                             21547
intravenous                                                                          3374
intradermal; percutaneous; subcutaneous                                              1315
respiratory (inhalation)                                                             1257
                                                                                    ...  
sublingual; sublingual; sublingual; sublingual                                          1
intracanalicular; ophthalmic                                                            1
retrobulbar; topical                                                                    1
intracavitary                                                                           1
infiltration; intra-articular; intralesional; intramuscular; perineural; topical        1
Name: ROUT

Cette colonne contient 180 différentes valeurs textuelles. Comme on peut le voir, différentes catégories peuvent être 
affectées au même objet. La colonne présente 1932 valeurs manquantes.

### Colonne STARTMARKETINGDATE
Les valeurs sont de type date, il n'y a aucune valeur manquante. 

### Colonne ENDMARKETINGDATE
Les valeurs sont de type date, il y a un grand nombre de valeurs manquantes. 

### Colonne MARKETINGCATEGORYNAME

In [19]:
product_data['MARKETINGCATEGORYNAME'].value_counts()
# %%
"""
Les valeurs sont de type textuelles, il y a 26 catégories différentes et ne présente aucune valeur manquante.  
"""
# %%
"""
### Colonne APPLICATIONNUMBER
"""
# %%
product_data['APPLICATIONNUMBER'].nunique()
# %%
"""
Cette colonne spécifie le numéro de série de la catégorie marketing. Le nombre de valeurs manquantes est élevé, et 
comme il y a un numéro de série pour chaque objet dans une catégorie, le nombre de valeurs différentes est également 
important.
"""

'\nCette colonne spécifie le numéro de série de la catégorie marketing. Le nombre de valeurs manquantes est élevé, et \ncomme il y a un numéro de série pour chaque objet dans une catégorie, le nombre de valeurs différentes est également \nimportant.\n'

### Colonne LABELERNAME

In [20]:
product_data['LABELERNAME'].nunique()

6098

In [21]:
product_data['LABELERNAME'][7291:7293]
# %%
"""
La colonne présente peu de valeurs manquantes (557). Si on remarque qu'il existe un nombre important de valeurs 
différentes, les données sont cependant inconsistantes.
"""

"\nLa colonne présente peu de valeurs manquantes (557). Si on remarque qu'il existe un nombre important de valeurs \ndifférentes, les données sont cependant inconsistantes.\n"

### Colonne SUBSTANCENAME

In [22]:
product_data['SUBSTANCENAME'].nunique()

8976

In [23]:
product_data['SUBSTANCENAME'][727:735]

727          felbamate
728          felbamate
729    cromolyn sodium
730           nabilone
731       carisoprodol
732       carisoprodol
733          torsemide
734      penicillamine
Name: SUBSTANCENAME, dtype: object

Cette colonne présente un nombre assez important de données manquantes (2309), ceux sont des données textuelles 
catégorielles. Or le nombre de catégories parait élevé, comme le montre le nombre de valeurs uniques. Chaque objet 
peut cependant présenté plusieurs catégories séparées par un ';'.

### Colonne ACTIVE_NUMERATOR_STRENGTH

In [24]:
product_data['ACTIVE_NUMERATOR_STRENGTH'][725:731]

725    137; 50
726        400
727        600
728        600
729         20
730          1
Name: ACTIVE_NUMERATOR_STRENGTH, dtype: object

Ceux sont des données numériques qui paraissent dupliquées pour le même objet.  

### Colonne ACTIVE_INGRED_UNIT

In [25]:
product_data['ACTIVE_INGRED_UNIT'][725:731]

725    ug/1; ug/1
726          mg/1
727          mg/1
728        mg/5ml
729         mg/ml
730          mg/1
Name: ACTIVE_INGRED_UNIT, dtype: object

Ceux sont des données textuelles catégorielles qui présentent l'unité de la colonne 'ACTIVE_INGRED_UNIT'. Les données
paraissent également dupliquées pour le même objet.

### Colonne PHARM_CLASSES

In [26]:
product_data['PHARM_CLASSES'].nunique()

1285

In [27]:
product_data['PHARM_CLASSES'][725]

'histamine h1 receptor antagonists [moa],histamine-1 receptor antagonist [epc],corticosteroid [epc],corticosteroid hormone receptor agonists [moa]'

Ceux sont des données textuelles catégorielles présentant plusieurs catégories pour un même objet. Il y a un grand 
nombre de valeurs manquantes.
"""
# %%
"""
### Colonne DEASCHEDULE

In [28]:
product_data['DEASCHEDULE'].value_counts()

cii     1802
civ     1749
ciii     479
cv       393
Name: DEASCHEDULE, dtype: int64

Cette colonne présente un nombre important de données manquantes. Ceux sont des données catégorielles, présentant 
seulement 4 catégories.

### Colonne NDC_EXCLUDE_FLAG

In [29]:
product_data['NDC_EXCLUDE_FLAG'].value_counts()

n    93238
Name: NDC_EXCLUDE_FLAG, dtype: int64

Cette colonne présente seulement une catégorie 'N'.

# 2. Relations entre attributs
## Informations communes
Les colonnes 'PRODUCTID' des tables 'package' et 'product' contiennent deux informations concaténées: l'id du produit 
ainsi que le contenu de leur colonne 'PRODUCTNDC', le code label et le code segment produit.  
Dans la documentation NDC, il est précisé que c'est pour prévenir le duplicata de lignes.

La colonne 'NDCPACKAGECODE' de la table 'package' contient deux informations concaténées: le code segment du package et 
le contenu de la colonne 'PRODUCTNDC', le code label et le code segment produit.

La colonne 'PACKAGEDESCRIPTION' de la table 'package' contient plusieurs informations concaténées. En plus des 
informations propres à la description du package, il y a dans la majorité des objets la valeur 'NDCPACKAGECODE' associée
.

La colonne 'APPLICATIONNUMBER' de la table 'product' présente la majorité du temps le contenu de la colonne 
'MARKETINGCATEGORYNAME' et spécifie son numéro de série.

Dans les deux tables, il existe des colonnes 'STARTMARKETINGDATE',  'ENDMARKETINGDATE' et 'NDCEXLUDEDFLAG'. 
Elles semblent présenter les mêmes informations.

## Corrélation
Pour la table 'product':
Il semble pouvoir exister une corrélation entre les attributs 'ROUTENAME' et 'DOSAGEFORMNAME' qui présentent des idées 
d'administration similaires. 
On peut également considérer l'existance d'une corrélation entre les modes d'administration
et les dosages du médicament, donc les attributs 'ROUTENAME', 'DOSAGEFORMNAME' et ceux 'ACTIVE_NUMERATOR_STRENGTH', 
'ACTIVE_INGRED_UNIT'.
L'attribut 'PHARM_CLASS' semble pouvoir être corrélé à l'attribut 'SUBSTANCENAME'.

# 3. Correction des incohérences
On élimine dans un premier temps les duplicata de valeurs dans les attributs 'ACTIVE_NUMERATOR_STRENGTH', 
'ACTIVE_INGRED_UNIT' de la table 'product'. 
## Table 'product'

In [30]:

# TODO: keep most frequent value
# dupl_val_cols = ['ACTIVE_NUMERATOR_STRENGTH', 'ACTIVE_INGRED_UNIT']
# for c in dupl_val_cols:
#     product_data[c] = product_data[c].replace(to_replace=r'\;.*', value='', regex=True)


Il existerait également une incohérence si l'attribut 'ENDMARKETINGDATE' est moins récent que le 'STARTMARKETINGDATE'.
On vérifie s'il en existe dans les tables 'product' et 'package'.

In [31]:

# conversion to datetime format
def date_convert():
    date_cols = ['STARTMARKETINGDATE', 'ENDMARKETINGDATE', 'LISTING_RECORD_CERTIFIED_THROUGH']
    for c in date_cols:
        product_data[c] = pd.to_datetime(product_data[c], errors='coerce', format='%Y%m%d')


if not product_encode_file_exist:
    time_methode(date_convert)

# compare STARTMARKETINGDATE and ENDMARKETINGDATE
# replace ENDMARKETINGDATE to NaT when incoherence
product_data.loc[
    (product_data['STARTMARKETINGDATE'] > product_data['ENDMARKETINGDATE']), 'ENDMARKETINGDATE'] = pd.NaT

Timing date_convert
Start time: 2020-03-08 03:18:39.667328
End time: 2020-03-08 03:18:39.782852
date_convert took: 0:00:00.115524


## Table 'package'
La colonne 'PACKAGEDESCRIPTION' contient beaucoup trop d'informations pour être exploitable. Tout d'abord, on garde 
seulement l'information du package le plus informatif (le dernier) car spécifie le volume le plus précis.
On supprime l'information dupliquée du 'NDCPACKAGECODE'. 
Enfin, on crée une colonne pour chaque information: 'PACKAGESIZE', 'PACKAGEUNIT' et 'PACKAGETYPE'.
On peut retirer la colonne 'PACKAGEDESCRIPTION' de la table.

In [32]:

if not package_encode_file_exist:
    # keep only most informative packaging and remove duplicate info NDCPACKAGECODE
    package_data['PACKAGEDESCRIPTION'] = package_data['PACKAGEDESCRIPTION'].replace(to_replace=r'.*(\>|\*\ ) |\(.*',
                                                                                    value='', regex=True)

    # split info into multiple columns
    search = {0: [], 1: [], 2: []}
    for values in package_data['PACKAGEDESCRIPTION']:
        s = re.search(r'(^\.?[0-9\.]+)\ (.*)\ in\ 1\ (.*)', values)
        for i in range(3):
            search[i].append(s.group(i + 1))

    for i, n in enumerate(['PACKAGESIZE', 'PACKAGEUNIT', 'PACKAGETYPE']):
        package_data[n] = search[i]

Traitement des colonnes 'STARTMARKETINGDATE', 'ENDMARKETINGDATE' similairement à la table 'product'.

In [33]:

if not package_encode_file_exist:
    # conversion to datetime format
    date_cols = ['STARTMARKETINGDATE', 'ENDMARKETINGDATE']
    for c in date_cols:
        package_data[c] = pd.to_datetime(package_data[c], errors='coerce', format='%Y%m%d')

    # compare STARTMARKETINGDATE and ENDMARKETINGDATE
    # replace ENDMARKETINGDATE to NaT when incoherence
    package_data.loc[
        (package_data['STARTMARKETINGDATE'] > package_data['ENDMARKETINGDATE']), 'ENDMARKETINGDATE'] = pd.NaT

# 4. Données manquantes
## Table 'package'
On s'intéresse aux données manquantes dans les colonnes PRODUCTID, PRODUCTNDC, NDCPACKAGECODE.

In [34]:
if not package_encode_file_exist:
    package_missing_ndcpackagecode = package_data.iloc[np.where(pd.isnull(package_data['NDCPACKAGECODE']))]
    values = package_missing_ndcpackagecode['PACKAGEDESCRIPTION'].str.extract(r'\((.*?)\).*')
    for index, row in values.iterrows():
        package_data.loc[index, 'NDCPACKAGECODE'] = row[0]

In [35]:
if not package_encode_file_exist:
    package_missing_productndc = package_data.iloc[np.where(pd.isnull(package_data['PRODUCTNDC']))]
    values = package_missing_productndc['NDCPACKAGECODE'].str.extract(r'^([\w]+-[\w]+)')
    for index, row in values.iterrows():
        package_data.loc[index, 'PRODUCTNDC'] = row[0]

In [36]:

if not package_encode_file_exist:
    # TODO : find a way to retrieve PRODUCTID from 'product' table
    package_missing_ndcproductid = package_data.iloc[np.where(pd.isnull(package_data['PRODUCTID']))]

Il existe des valeurs manquantes pour les colonnes 'STARTMARKETINGDATE' et 'ENDMARKETINGDATE' dans la table 'package'
mais on choisit de ne pas les compléter car on ne peut effectuer d'estimation précise. 

## Table 'product'

# 5. Duplications données

In [37]:

# TODO: drop column PACKAGEDESCRIPTION

# Transformation en données numériques (après question 8)
## Table 'package'

## Table 'product'

In [38]:

# %%
# TODO: hash PROPRIETARYNAME NONPROPRIETARYNAME LABELERNAME PROPRIETARYNAMESUFFIX
# TODO: separate and hash SUBSTANCENAME PHARM_CLASSES
# TODO : split ACTIVE_INGRED_UNIT by '/' (nan others), then one hot each col

# TODO: ideas?? APPLICATIONNUMBER
# %%
# TODO : analysis ratio per category

## Encodage onehot

In [39]:

# Call and time onehot encoding for all predefined columns
if not os.path.isdir(encoder_dir):
    os.mkdir(encoder_dir)
if not product_encode_file_exist:
    for header in product_headers_to_encode:
        enc_dic[header] = time_methode(onehot_encode, header, **(dict(table=product_data, header=header)))
        pickle.dump(enc_dic[header], open(encoder_dir + '{}_data_encoder.pkl'.format(header), 'wb'), pickle.HIGHEST_PROTOCOL)

if not package_encode_file_exist:
    for header in package_headers_to_encode:
        enc_dic[header] = time_methode(onehot_encode, header, **(dict(table=package_data, header=header)))
        pickle.dump(enc_dic[header], open(encoder_dir + '{}_data_encoder.pkl'.format(header), 'wb'), pickle.HIGHEST_PROTOCOL)

if not os.path.isdir(encoding_dir):
    os.mkdir(encoding_dir)
# Prints out encding of each category for a given column in a txt file
for header, enc in enc_dic.items():
    file = open(encoding_dir + 'Encoding_{}.txt'.format(header), 'w')
    for category in enc.categories_[0]:
        tmp_str = str(enc.transform([[category]]).toarray())
        tmp_str = category + ' ' * (40 - len(category)) + tmp_str.replace('\n', '\n' + ' ' * 40) + '\n'
        file.write(tmp_str)
    file.close()

# Save transformed data to file
if not product_encode_file_exist:
    time_methode(product_data.to_csv, **(dict(path_or_buf= encoded_product_file,
                                              index= False,
                                              sep= separ,
                                              encoding= target_encoding,
                                              quoting= csv.QUOTE_NONNUMERIC)))

if not product_encode_file_exist:
    time_methode(package_data.to_csv, **(dict(path_or_buf=encoded_package_file,
                                              index=False,
                                              sep=separ,
                                              encoding=target_encoding,
                                              quoting=csv.QUOTE_NONNUMERIC)))

Timing onehot_encode
PRODUCTTYPENAME
Start time: 2020-03-08 03:19:34.203847
|==================================================| 100.0% -> Done
End time: 2020-03-08 03:20:15.302815
onehot_encode took: 0:00:41.098968
Timing onehot_encode
ROUTENAME
Start time: 2020-03-08 03:20:15.303710
|==================================================| 100.0% -> Done
End time: 2020-03-08 03:21:01.192735
onehot_encode took: 0:00:45.889025
Timing onehot_encode
DOSAGEFORMNAME
Start time: 2020-03-08 03:21:01.194065
|==================================================| 100.0% -> Done
End time: 2020-03-08 03:21:58.522046
onehot_encode took: 0:00:57.327981
Timing onehot_encode
MARKETINGCATEGORYNAME
Start time: 2020-03-08 03:21:58.522885
|==================================================| 100.0% -> Done
End time: 2020-03-08 03:22:38.519683
onehot_encode took: 0:00:39.996798
Timing onehot_encode
ACTIVE_NUMERATOR_STRENGTH
Start time: 2020-03-08 03:22:38.520477
|==================================================

## Résultats

In [40]:
print('Encoded product data:')
print(product_data)
product_data

Encoded product data:
                                            PRODUCTID PRODUCTNDC  \
0                                                 NaN  0002-0800   
1                                                 NaN  0002-1200   
2                                                 NaN  0002-1433   
3                                                 NaN  0002-1434   
4                                                 NaN  0002-1436   
...                                               ...        ...   
93233  99207-465_7578e84a-41ed-498d-8c2b-56a9931679db  99207-465   
93234  99207-466_7578e84a-41ed-498d-8c2b-56a9931679db  99207-466   
93235  99207-467_7578e84a-41ed-498d-8c2b-56a9931679db  99207-467   
93236  99207-525_d47eda34-3952-463c-9597-4225a19dbf13  99207-525   
93237  99207-850_76ac026f-e6f1-4d1f-8144-9b5492e2d1bb  99207-850   

               PRODUCTTYPENAME  PROPRIETARYNAME PROPRIETARYNAMESUFFIX  \
0      [[0, 1, 0, 0, 0, 0, 0]]  sterile diluent                   NaN   
1      [[0, 0, 

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,NaN,0002-0800,"[[0, 1, 0, 0, 0, 0, 0]]",sterile diluent,NaN,diluent,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1987-07-10,NaT,"[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]",nda018781,10,water,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",NaN,NaN,n,2020-12-31
1,NaN,0002-1200,"[[0, 0, 1, 0, 0, 0, 0]]",amyvid,NaN,florbetapir f 18,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2012-06-01,NaT,"[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]",nda202008,10,florbetapir f-18,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","radioactive diagnostic agent [epc],positron em...",NaN,n,2021-12-31
2,NaN,0002-1433,"[[0, 0, 1, 0, 0, 0, 0]]",trulicity,NaN,dulaglutide,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2014-09-18,NaT,"[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]",bla125469,10,dulaglutide,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","glp-1 receptor agonist [epc],glucagon-like pep...",NaN,n,2020-12-31
3,NaN,0002-1434,"[[0, 0, 1, 0, 0, 0, 0]]",trulicity,NaN,dulaglutide,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2014-09-18,NaT,"[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]",bla125469,10,dulaglutide,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","glp-1 receptor agonist [epc],glucagon-like pep...",NaN,n,2020-12-31
4,NaN,0002-1436,"[[0, 0, 1, 0, 0, 0, 0]]",emgality,NaN,galcanezumab,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2018-09-27,NaT,"[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]",bla761063,10,galcanezumab,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",NaN,NaN,n,2020-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93233,99207-465_7578e84a-41ed-498d-8c2b-56a9931679db,99207-465,"[[0, 0, 1, 0, 0, 0, 0]]",solodyn,NaN,minocycline hydrochloride,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",NaN,2010-09-27,NaT,"[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]",nda050808,valeant pharmaceuticals north america llc,minocycline hydrochloride,NaN,NaN,"tetracycline-class drug [epc],tetracyclines [cs]",NaN,n,2020-12-31
93234,99207-466_7578e84a-41ed-498d-8c2b-56a9931679db,99207-466,"[[0, 0, 1, 0, 0, 0, 0]]",solodyn,NaN,minocycline hydrochloride,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",NaN,2010-09-27,NaT,"[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]",nda050808,valeant pharmaceuticals north america llc,minocycline hydrochloride,NaN,NaN,"tetracycline-class drug [epc],tetracyclines [cs]",NaN,n,2020-12-31
93235,99207-467_7578e84a-41ed-498d-8c2b-56a9931679db,99207-467,"[[0, 0, 1, 0, 0, 0, 0]]",solodyn,NaN,minocycline hydrochloride,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",NaN,2010-09-27,NaT,"[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]",nda050808,valeant pharmaceuticals north america llc,minocycline hydrochloride,NaN,NaN,"tetracycline-class drug [epc],tetracyclines [cs]",NaN,n,2020-12-31
93236,99207-525_d47eda34-3952-463c-9597-4225a19dbf13,99207-525,"[[0, 0, 1, 0, 0, 0, 0]]",vanos,NaN,fluocinonide,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...",NaN,2006-03-13,NaT,"[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]",nda021758,valeant pharmaceuticals north america llc,fluocinonide,NaN,NaN,"corticosteroid [epc],corticosteroid hormone re...",NaN,n,2020-12-31


In [41]:
print('Encoded packaging data:')
print(package_data)
package_data

Encoded packaging data:
                                             PRODUCTID PRODUCTNDC  \
0       0002-0800_94c48759-29bb-402d-afff-9a713be11f0e  0002-0800   
1       0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932  0002-1200   
2       0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932  0002-1200   
3       0002-1433_42a80046-fd68-4b80-819c-a443b7816edb  0002-1433   
4       0002-1433_42a80046-fd68-4b80-819c-a443b7816edb  0002-1433   
...                                                ...        ...   
173882  99207-525_d47eda34-3952-463c-9597-4225a19dbf13  99207-525   
173883  99207-525_d47eda34-3952-463c-9597-4225a19dbf13  99207-525   
173884  99207-525_d47eda34-3952-463c-9597-4225a19dbf13  99207-525   
173885  99207-850_76ac026f-e6f1-4d1f-8144-9b5492e2d1bb  99207-850   
173886  99207-850_76ac026f-e6f1-4d1f-8144-9b5492e2d1bb  99207-850   

       NDCPACKAGECODE           PACKAGEDESCRIPTION STARTMARKETINGDATE  \
0        0002-0800-01              10 ml in 1 vial         1987-07-10   
1

,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE,PACKAGESIZE,PACKAGEUNIT,PACKAGETYPE
0,0002-0800_94c48759-29bb-402d-afff-9a713be11f0e,0002-0800,0002-0800-01,10 ml in 1 vial,1987-07-10,NaT,n,n,10,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-30,"30 ml in 1 vial, multi-dose",2012-06-01,NaT,n,n,30,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-50,"50 ml in 1 vial, multi-dose",2012-06-01,NaT,n,n,50,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-61,.5 ml in 1 syringe,2014-11-07,NaT,n,y,.5,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-80,.5 ml in 1 syringe,2014-11-07,NaT,n,n,.5,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...,...,...,...,...,...,...,...,...,...
173882,99207-525_d47eda34-3952-463c-9597-4225a19dbf13,99207-525,99207-525-10,120 g in 1 tube,2006-03-13,NaT,n,n,120,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
173883,99207-525_d47eda34-3952-463c-9597-4225a19dbf13,99207-525,99207-525-30,30 g in 1 tube,2006-03-13,NaT,n,n,30,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
173884,99207-525_d47eda34-3952-463c-9597-4225a19dbf13,99207-525,99207-525-60,60 g in 1 tube,2006-03-13,NaT,n,n,60,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
173885,99207-850_76ac026f-e6f1-4d1f-8144-9b5492e2d1bb,99207-850,99207-850-02,2 g in 1 tube,2013-11-14,NaT,n,y,2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
